In [ ]:
# trial for fft analysis

In [1]:
print("Loading Libraries...")
import os
import csv
import platform
import numpy as np
from scipy.fftpack import fft
from scipy.fftpack import fftfreq
import pandas as pd
import dask.dataframe as dd
import math
import matplotlib.pyplot as plt
import timeit
print("Loaded Libraries...")

Loading Libraries...
Loaded Libraries...


In [3]:
#Local
print("Loading directories..")
#path_data = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01'
#path_post = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post'
path_acu = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-tip-post/acu'
#path_plots = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/plots'
path_signal = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-tip-post/signal'
path_rms = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-tip-post/rms'
valid = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-tip-post/valid'
print("Loaded directories...")

Loading directories..
Loaded directories...


In [5]:
#PUT Workstation
print("Loading directories..") 
#path_data = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01'
#path_post = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post'
path_acu = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/acu'
#path_plots = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/plots'
path_signal = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/signal'
#path_rms = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/rms'
print("Loaded directories...")

Loading directories..
Loaded directories...


In [ ]:
def FFT_values(x):
    #T = 0.050150  # Duration in seconds
    T = 0.000060  # Duration in seconds
    Fs = 1000000  # Sampling frequency

    # Time domain signal
    #t = np.arange(0, T*Fs)/Fs
    x = np.asarray(x)
    N = x.size

    # DFT
    X = np.fft.fft(x)
    #X_db = 20*np.log10(2*np.abs(X)/N)
    #f = np.fft.fftfreq(N, 1/Fs)
    #f = np.arange(0, N)*Fs/N
    #return np.concatenate([X, X_db])
    return np.concatenate([X])

In [ ]:
def FFT_db(x):
    #T = 0.050150  # Duration in seconds
    T = 0.000114  # Duration in seconds
    #f0 = 100  # Fundamental frequency
    Fs = 25000  # Sampling frequency

    # Time domain signal
    t = np.arange(0, T*Fs)/Fs
    x = np.asarray(x, np.float)
    N = x.size

    # DFT
    X = np.fft.fft(x)
    X_db = 20*np.log10(2*np.abs(X)/N)
    #f = np.fft.fftfreq(N, 1/Fs)
    #f = np.arange(0, N)*Fs/N
    #return np.concatenate([X, X_db])
    return np.concatenate([X_db])

In [ ]:
def FFT_freq(x):
    #T = 0.050150  # Duration in seconds
    T = 0.000114  # Duration in seconds
    #f0 = 100  # Fundamental frequency
    Fs = 25000  # Sampling frequency
    
    # Time domain signal
    t = np.arange(0, T*Fs)/Fs
    x = np.asarray(x, np.float)
    N = x.size
    
    f = np.arange(0, N)*Fs/N
    return np.concatenate([f])

In [ ]:
def batch_fft(signal):
    #T = 0.050150  # Duration in seconds
    T = 0.000060  # Duration in seconds
    #f0 = 100  # Fundamental frequency
    Fs = 1000000  # Sampling frequency
    
    signal = np.asarray(signal)
    N = signal.size    
    
    X_np = np.fft.fft(signal)
    X_np_mag = 2*np.sqrt(X_np.real[:N//2]**2 + X_np.imag[:N//2]**2)/N
    X_np_phase = np.arctan(X_np.imag[:N//2]/X_np.real[:N//2])
    #f_np = np.arange(0, N//2)*Fs/N
    
    node_fft = [X_np_mag, X_np_phase]
    
    return node_fft

In [ ]:
def scipy_fft(y):
    y = np.asarray(y)
    N = len(y)
    T = 1.0 / 100000
    x = np.linspace(0.0, N*T, N)
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N)
    return [xf, yf]

In [6]:
print("Loading batch data...")
os.chdir(path_acu)
batch_pressure = dd.read_csv('int-*', delimiter=",", decimal='.',usecols=["nodenumber", "sound-pressure"])
batch_pressure = batch_pressure.set_index("nodenumber")
print("Batch data done...")

Loading batch data...


KeyboardInterrupt: 

In [ ]:
fft_batch = batch_pressure.groupby('nodenumber').apply(lambda x: fft(x), meta=('node-fft-data', 'f8')).compute()

In [ ]:
fft_batch.to_csv(str('int-tip_fft_test.dat'), sep=",")

In [ ]:
fft_batch.iloc[0]#[:30]

In [ ]:
f_np = np.arange(0, 30)*100000/60

In [ ]:
plt.plot(f_np[:60//4], fft_batch.iloc[0][0][0][:60//4])
#plt.plot(f_np[:N//4], X_np_phase[:N//4])
#plt.plot(f_sp.real, X_sp.real)
plt.grid()
plt.show()

In [ ]:
f_np.size

In [ ]:
#fft_values = pd.DataFrame(batch_pressure.groupby('nodenumber').apply(lambda x: FFT_values(x), meta=('sound-pressure', 'f8')).compute()).sort_index(axis=0, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True)
fft_db = pd.DataFrame(batch_pressure.groupby('nodenumber').apply(lambda x: FFT_db(x), meta=('sound-pressure', 'f8')).compute()).sort_index(axis=0, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True)
fft_freq = pd.DataFrame(batch_pressure.groupby('nodenumber').apply(lambda x: FFT_freq(x), meta=('sound-pressure', 'f8')).compute()).sort_index(axis=0, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True)

In [ ]:
os.chdir(valid)
fft_db.to_csv(str('int-01_fft_db.dat'), sep=",")
fft_db

In [ ]:
os.chdir(valid)
fft_freq.to_csv(str('int-01_fft_freq.dat'), sep=",")
fft_freq

In [ ]:
j = 1
node_DB = fft_db.loc[j].values[0]
node_FREQ = fft_freq.loc[j].values[0]
node_DB_max = np.max(node_DB)
node_DB_deviation = np.divide((np.abs(np.mean(node_DB))) - np.abs(node_DB_max), np.std(node_DB))
node_FREQ_max = node_FREQ[np.flatnonzero(node_DB == np.max(node_DB))][0]
node_fft_max = {'nodenumber': [j], 'node_DB_max': [node_DB_max], 'node_DB_deviation': [node_DB_deviation], 'node_FREQ_max': [node_FREQ_max]}
node_fft_max = pd.DataFrame(data = node_fft_max)
#node_fft_max.columns = ['nodenumber', 'node_DB_max', 'node_DB_deviation', 'node_FREQ_max']
#node_fft_max_i.append(node_fft_max)

nodelist = list(range(2, 38138+1))
for i in nodelist:
    node_DB = fft_db.loc[i].values[0]
    node_FREQ = fft_freq.loc[i].values[0]
    node_DB_max = np.max(node_DB)
    node_DB_deviation = np.divide((np.abs(np.mean(node_DB)) - np.abs(node_DB_max)), np.std(node_DB))
    node_FREQ_max = node_FREQ[np.flatnonzero(node_DB == np.max(node_DB))][0]
    node_fft_max_i = {'nodenumber': [i], 'node_DB_max': [node_DB_max], 'node_DB_deviation': [node_DB_deviation], 'node_FREQ_max': [node_FREQ_max]}
    node_fft_max_i = pd.DataFrame(data = node_fft_max_i)
    #node_fft_max_i.columns = ['nodenumber', 'node_DB_max', 'node_DB_deviation', 'node_FREQ_max']
    node_fft_max = node_fft_max.append(node_fft_max_i, ignore_index=True)
    print(str('Node ' + str(i) + ' done...'))

In [ ]:
node_fft_max = node_fft_max.set_index('nodenumber')
node_fft_max

In [ ]:
i = 3
node_DB = fft_db.loc[i].values[0]
node_DB

In [ ]:
DB = fft_db.loc[2].values[0]
#DB.to_csv(str('int-01_DB.dat'), sep=",")
DB
#FREQ = np.asarray(fft_freq.loc[2])[0]
#FREQ
#VAL = fft_values.loc[2]

In [ ]:
np.std(DB)

In [ ]:
np.abs(np.mean(DB))

In [ ]:
np.max(DB)

In [ ]:
dupa = np.flatnonzero(DB == np.max(DB))[0]

In [ ]:
FREQ = fft_freq.loc[2].values[0]
#FREQ.to_csv(str('int-01_FREQ.dat'), sep=",")
FREQ[dupa]

In [ ]:
FREQ = np.asarray(FREQ)
FREQ = FREQ[0]
FREQ

In [ ]:
DB.to_csv(str('int-01_DB.dat'), sep=",")
DB

In [ ]:
VAL

In [ ]:
DBRA = DB.values[0]
DBRA.size

In [ ]:
plt.plot(FREQ, DB)
plt.grid()
plt.show()

In [ ]:
# code validation

In [ ]:
def FFT(x):
    """A recursive implementation of the 1D Cooley-Tukey FFT"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 32:  # this cutoff should be optimized
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N / 2] * X_odd,
                               X_even + factor[N / 2:] * X_odd])

In [ ]:
#NUMPY
#signal props
T_test = 0.05015  # Duration in seconds
N = 50150 # Number of sample points
Fs = N / T_test  # Sampling frequency [Hz]
T = 1.0 / Fs # sample spacing

# Time domain signal for test sinusoid
f0_test = 1000  # Fundamental frequency
t_test = np.arange(0, N)/Fs
#N_test = t_test.size
#signal = np.sin(2*np.pi*f0_test*t_test) + 0.5*np.sin(2*np.pi*2*f0_test*t_test) + 0.25*np.sin(2*np.pi*4*f0_test*t_test)

signal = np.sin(2*np.pi*f0_test*t_test)
i=3
while i<=10:
    f = np.sin(2*np.pi*i*f0_test*t_test)/i
    signal = signal + f
    i = i+2

In [ ]:
# DFT Numpy
X_np = np.fft.fft(signal)
#X_db_np = 20*np.log10(2*np.abs(X_np)/N_test)
X_np_mag = 2*np.sqrt(X_np.real[:N//2]**2 + X_np.imag[:N//2]**2)/N
X_np_phase = np.arctan(X_np.imag[:N//2]/X_np.real[:N//2])
f_np = np.arange(0, N//2)*Fs/N
#f_np = np.fft.fftfreq(N, 1/Fs)
#f_np = np.arange(0, N_test)*Fs/N

In [ ]:
# DFT Scipy
X_sp = fft(signal)
#X_db_np = 20*np.log10(2*np.abs(X_np)/N_test)
X_sp_mag = 2*np.sqrt(X_sp.real[:N//2]**2 + X_sp.imag[:N//2]**2)/N
X_sp_phase = np.arctan(X_sp.imag[:N//2]/X_sp.real[:N//2])
f_sp = np.arange(0, N//2)*Fs/N
#f_np = np.fft.fftfreq(N, 1/Fs)
#f_np = np.arange(0, N_test)*Fs/N

In [ ]:
'''
#Scipy
#x = np.linspace(0.0, N*T, N)
#y = np.sin(f0_test * 2.0*np.pi*x)

# DFT
X_sp = fft(signal)
X_db_sp = 20*np.log10(2*np.abs(X_sp)/N)
#xf = np.linspace(0.0, 1.0/(2.0*T), N)
f_sp = fftfreq(N, T)


plt.plot(absf_np.real, X_np.real)
plt.plot(f_sp.real, X_sp.real)
plt.grid()
plt.show()'''

In [ ]:
plt.figure(figsize=(20, 5), dpi=300)
plt.plot(f_np[:N//40], X_np_mag[:N//40])
plt.plot(f_sp[:N//40], X_sp_mag[:N//40])
#plt.plot(f_sp.real, X_sp.real)
plt.grid()
plt.show()

In [ ]:
plt.plot(signal[:1001])
#plt.plot(abs(f_sp), X_db_sp)
plt.grid()
plt.show()

In [ ]:
len(X_np)

In [ ]:
plt.plot(t_test, x_test)
#plt.plot(abs(xf), 20*np.log10(2*np.abs(yf)/N))
plt.grid()
plt.show()

In [ ]:
# Number of sample points
N = 50150
# sample spacing
T = 1.0 / 100000.0
x = np.linspace(0.0, N*T, N)
y = np.sin(1000.0 * 2.0*np.pi*x)
yf = fft(y)
#xf = np.linspace(0.0, 1.0/(2.0*T), N)
xf = fftfreq(N, T)
#plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.plot(xf, 20*np.log10(2*np.abs(yf)/N))
plt.grid()
plt.show()

In [ ]:
x

In [ ]:
t_test

In [ ]:
x_test

In [ ]:
X_test

In [7]:
#fft.py
print("Loading Libraries...")
import os
import csv
import platform
import numpy as np
from scipy.fftpack import fft
import pandas as pd
import dask.dataframe as dd
import math
#import matplotlib.pyplot as plt
print("Loaded Libraries...")

print("Starting code...")

'''
#PUT Workstation
print("Loading directories..") 
#path_data = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01'
#path_post = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post'
path_acu = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/acu'
#path_plots = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/plots'
#path_signal = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/signal'
path_fft = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/fft'
print("Loaded directories...")
'''
#Local
print("Loading directories..")
#path_data = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01'
#path_post = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post'
path_acu = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-tip-post/acu'
#path_plots = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/plots'
path_fft = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/fft'
print("Loaded directories...")

'''
#PLGRID
print("Loading directories..")
#path_data = '/net/scratch/people/plgmosieznyj/SRS_v02/noise-data/int-01'
#path_post = '/net/scratch/people/plgmosieznyj/SRS_v02/noise-data/int-01-post'
path_acu = '/net/scratch/people/plgmosieznyj/SRS_v02/noise-data/int-01-post/acu'
#path_plots = '/net/scratch/people/plgmosieznyj/SRS_v02/noise-data/int-01-post/plots'
path_fft = '/net/scratch/people/plgmosieznyj/SRS_v02/noise-data/fft'
print("Loaded directories...")
'''
print("Loading batch data...")
os.chdir(path_acu)
batch_pressure = dd.read_csv('*.dat', delimiter=",", decimal='.',usecols=["nodenumber", "sound-pressure"]).set_index("nodenumber")
print("Batch data done...")

print("Calculating FFT...") 
batch_fft = batch_pressure.groupby('nodenumber').apply(lambda x: fft(x), meta=('node-fourier-series', 'f8')).compute()
print("FFT Done..") 

print("Saving FFT to dataframe...")
#node_fft_max.set_index('nodenumber')
os.chdir(path_fft)
batch_fft.to_csv(str('int-01_fft.dat'), sep=",")
print("Dataframe saved...")
print("Script completed...")

Loading Libraries...
Loaded Libraries...
Starting code...
Loading directories..
Loaded directories...
Loading batch data...
Batch data done...
Calculating FFT...
FFT Done..
Saving FFT to dataframe...
Dataframe saved...
Script completed...
